In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import warnings
warnings.simplefilter(action='ignore')

In [56]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
data = pd.read_csv(url, names=columns)

X = data.drop('species', axis=1).values
y = data['species'].values

y = LabelEncoder().fit_transform(y)
y = torch.tensor(y, dtype=torch.long)
X = torch.tensor(X, dtype=torch.float32)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

In [57]:
class neuroverkko(nn.Module):
    def __init__(self):
        super(neuroverkko, self).__init__()
        self.hidden = nn.Linear(4, 10)  # Piilotettu kerros, jossa 10 neuronia
        self.output = nn.Linear(10, 3)  # Ulostulokerros, jossa 3 neuronia

    def forward(self, x):
        x = torch.relu(self.hidden(x))  # Aktivointifunktio piilotetussa kerroksessa
        x = self.output(x)  # Ulostulo ilman softmaxia (CrossEntropyLoss hoitaa tämän)
        return x

model = neuroverkko()

In [58]:
# Tappiofunktio ja optimoija
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Koulutussilmukka
for epoch in range(50):  # Kouluta 50 kertaa
    model.train()  # Laita malli koulutustilaan
    optimizer.zero_grad()  # Nollaa vanhat gradit
    outputs = model(X_train)  # Syötä koulutusdata mallille
    loss = criterion(outputs, y_train)  # Laske tappio
    loss.backward()  # Laske gradit
    optimizer.step()  # Päivitä painot
    
    # Lasketaan tarkkuus validointidatalla
    model.eval()  # Laita malli arviointitilaan
    with torch.no_grad():  # Ei lasketa gradientteja
        val_outputs = model(X_val)
        _, predicted = torch.max(val_outputs, 1)  # Valitsee suurimman ennusteen
        accuracy = (predicted == y_val).sum().item() / y_val.size(0) * 100

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.2f}%")


Epoch 1, Loss: 1.1407, Validation Accuracy: 36.67%
Epoch 2, Loss: 1.1043, Validation Accuracy: 36.67%
Epoch 3, Loss: 1.0667, Validation Accuracy: 36.67%
Epoch 4, Loss: 1.0279, Validation Accuracy: 36.67%
Epoch 5, Loss: 0.9932, Validation Accuracy: 36.67%
Epoch 6, Loss: 0.9644, Validation Accuracy: 36.67%
Epoch 7, Loss: 0.9389, Validation Accuracy: 43.33%
Epoch 8, Loss: 0.9143, Validation Accuracy: 70.00%
Epoch 9, Loss: 0.8895, Validation Accuracy: 70.00%
Epoch 10, Loss: 0.8641, Validation Accuracy: 70.00%
Epoch 11, Loss: 0.8374, Validation Accuracy: 70.00%
Epoch 12, Loss: 0.8133, Validation Accuracy: 70.00%
Epoch 13, Loss: 0.7931, Validation Accuracy: 70.00%
Epoch 14, Loss: 0.7739, Validation Accuracy: 70.00%
Epoch 15, Loss: 0.7545, Validation Accuracy: 70.00%
Epoch 16, Loss: 0.7345, Validation Accuracy: 70.00%
Epoch 17, Loss: 0.7147, Validation Accuracy: 73.33%
Epoch 18, Loss: 0.6941, Validation Accuracy: 80.00%
Epoch 19, Loss: 0.6729, Validation Accuracy: 80.00%
Epoch 20, Loss: 0.652